In [2]:
import pygame
import numpy as np
import random
from sys import exit
from Agent.TriviaLC import triviaAgent
from Agent.HangmanLC import HangMan

pygame 2.5.2 (SDL 2.28.3, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


Ashkan

In [ ]:
# Initialize agents
trivia_agent = triviaAgent()
hangman_agent = HangMan()

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((800, 800))
pygame.display.set_caption("Maropelleh (Snakes, Ladders, Trivia, Hangman)")
clock = pygame.time.Clock()

# Load assets
sky_background = pygame.image.load('graphics/blue-sky.png')

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
GRAY = (200, 200, 200)
YELLOW = (255, 255, 0)
LIGHT_YELLOW = (255, 255, 200)
ORANGE = (255, 200, 0)

# Game constants
BOARD_SIZE = 10
CELL_SIZE = 60
GRID_OFFSET = 100
PLAYER_SIZE = 15
BUTTON_WIDTH = 150
BUTTON_HEIGHT = 50

# Game state
player1_pos = 0
player2_pos = 0
current_player = 1
game_over = False
winner = None
message = "Player 1's turn. Press the button to roll!"
message_timer = 0
message_duration = 120

# Mini-game modal
active_modal = None  # 'trivia', 'hangman', or None
modal_data = None
user_answer = ""

# Snakes and ladders
snakes = {}
ladders = {}
for _ in range(5):
    start = random.randint(20, 98)
    end = random.randint(1, start - 1)
    snakes[end] = start
for _ in range(5):
    start = random.randint(1, 80)
    end = random.randint(start + 1, 99)
    ladders[start] = end

# Mini-game tiles
total_cells = list(range(1, 100))
random.shuffle(total_cells)
trivia_cells = set(total_cells[:15])
hangman_cells = set(total_cells[15:30])

# Fonts
font = pygame.font.SysFont(None, 36)
small_font = pygame.font.SysFont(None, 24)


def get_cell_position(pos):
    row = (pos - 1) // BOARD_SIZE
    col = (pos - 1) % BOARD_SIZE
    x = col * CELL_SIZE + GRID_OFFSET + CELL_SIZE // 2
    y = (BOARD_SIZE - 1 - row) * CELL_SIZE + GRID_OFFSET + CELL_SIZE // 2
    return x, y


def draw_board():
    screen.fill(WHITE)
    screen.blit(sky_background, (0, 0))
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            cell_num = row * BOARD_SIZE + col + 1
            x = col * CELL_SIZE + GRID_OFFSET
            y = (BOARD_SIZE - 1 - row) * CELL_SIZE + GRID_OFFSET
            color = BLUE if cell_num in trivia_cells else YELLOW if cell_num in hangman_cells else LIGHT_YELLOW if (row + col) % 2 == 0 else ORANGE
            pygame.draw.rect(screen, color, (x, y, CELL_SIZE, CELL_SIZE))
            pygame.draw.rect(screen, BLACK, (x, y, CELL_SIZE, CELL_SIZE), 1)
            text = font.render(str(cell_num), True, BLACK)
            screen.blit(text, text.get_rect(center=(x + CELL_SIZE//2, y + CELL_SIZE//2)))
            if cell_num in trivia_cells:
                screen.blit(small_font.render("T", True, WHITE), (x + CELL_SIZE//2 - 5, y + CELL_SIZE//2 + 15))
            elif cell_num in hangman_cells:
                screen.blit(small_font.render("H", True, BLACK), (x + CELL_SIZE//2 - 5, y + CELL_SIZE//2 + 15))


def draw_ladder(start, end):
    x1, y1 = get_cell_position(start)
    x2, y2 = get_cell_position(end)
    steps = 5
    dx, dy = (x2 - x1) / steps, (y2 - y1) / steps
    offset = 5
    pygame.draw.line(screen, GREEN, (x1 - offset, y1), (x2 - offset, y2), 3)
    pygame.draw.line(screen, GREEN, (x1 + offset, y1), (x2 + offset, y2), 3)
    for i in range(steps + 1):
        rx = x1 + dx * i
        ry = y1 + dy * i
        pygame.draw.line(screen, GREEN, (rx - offset, ry), (rx + offset, ry), 2)


def draw_snake(start, end):
    x1, y1 = get_cell_position(start)
    x2, y2 = get_cell_position(end)
    mid_x = (x1 + x2) // 2
    mid_y = (y1 + y2) // 2
    points = [(x1, y1), (mid_x + 20, mid_y - 30), (mid_x - 20, mid_y + 30), (x2, y2)]
    pygame.draw.lines(screen, RED, False, points, 4)
    pygame.draw.polygon(screen, RED, [(x1, y1), (x1 - 5, y1 - 10), (x1 + 5, y1 - 10)])


def draw_snakes_and_ladders():
    for start, end in ladders.items():
        draw_ladder(start, end)
    for end, start in snakes.items():
        draw_snake(start, end)


def draw_players():
    pygame.draw.circle(screen, RED, (get_cell_position(player1_pos + 1)[0] - 10, get_cell_position(player1_pos + 1)[1]), PLAYER_SIZE)
    pygame.draw.circle(screen, BLUE, (get_cell_position(player2_pos + 1)[0] + 10, get_cell_position(player2_pos + 1)[1]), PLAYER_SIZE)


def draw_button():
    rect = pygame.Rect(330, 720, BUTTON_WIDTH, BUTTON_HEIGHT)
    pygame.draw.rect(screen, YELLOW, rect)
    pygame.draw.rect(screen, BLACK, rect, 2)
    screen.blit(font.render("Roll Dice", True, BLACK), font.render("Roll Dice", True, BLACK).get_rect(center=rect.center))
    return rect


def draw_game_info():
    screen.blit(font.render(f"Current Player: {current_player}", True, BLACK), (20, 20))
    if game_over:
        screen.blit(font.render(f"Player {winner} wins!", True, BLACK), (20, 50))
    elif message_timer > 0:
        screen.blit(font.render(message, True, BLACK), (20, 70))


def draw_modal():
    if active_modal and modal_data:
        rect = pygame.Rect(100, 200, 600, 400)
        pygame.draw.rect(screen, GRAY, rect)
        pygame.draw.rect(screen, BLACK, rect, 3)
        lines = modal_data.split("\n")
        for i, line in enumerate(lines):
            screen.blit(small_font.render(line.strip(), True, BLACK), (rect.x + 20, rect.y + 30 + 25 * i))
        if active_modal == "trivia":
            screen.blit(small_font.render(f"Your answer: {user_answer}", True, BLUE), (rect.x + 20, rect.y + 330))
        screen.blit(small_font.render("Press ENTER to continue...", True, BLACK), (rect.x + 20, rect.y + 360))


def roll_dice():
    return random.randint(1, 6)


def move_player(pos, steps):
    new_pos = pos + steps
    if new_pos in ladders: new_pos = ladders[new_pos]
    if new_pos in snakes: new_pos = snakes[new_pos]
    return min(new_pos, 99), f"Player {current_player} moved to {new_pos+1}"


def trigger_mini_game(cell):
    global active_modal, modal_data
    if cell in trivia_cells:
        active_modal = "trivia"
        topics = ['Sports', 'literature', 'Movies', 'Celebrities', 'Music', 'general knowledge']
        modal_data = trivia_agent.envoke(topics[random.randint(0,len(topics))])
    elif cell in hangman_cells:
        active_modal = "hangman"
        modal_data = hangman_agent.envoke()

# Main Loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit(); exit()
        if active_modal == "trivia":
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_RETURN:
                    active_modal = None
                elif event.key == pygame.K_BACKSPACE:
                    user_answer = user_answer[:-1]
                    trivia_agent.check_answer(user_answer)
                elif event.unicode.isprintable():
                    user_answer += event.unicode
        elif active_modal:
            if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                active_modal = None
        elif event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            if draw_button().collidepoint(pygame.mouse.get_pos()):
                dice = roll_dice()
                if current_player == 1:
                    player1_pos, message = move_player(player1_pos, dice)
                    if player1_pos == 99: game_over = True; winner = 1
                    trigger_mini_game(player1_pos + 1)
                else:
                    player2_pos, message = move_player(player2_pos, dice)
                    if player2_pos == 99: game_over = True; winner = 2
                    trigger_mini_game(player2_pos + 1)
                message_timer = message_duration
                current_player = 3 - current_player
    if message_timer > 0:
        message_timer -= 1

    draw_board()
    draw_snakes_and_ladders()
    draw_players()
    draw_game_info()
    draw_button()
    draw_modal()
    pygame.display.update()
    clock.tick(60)

2025-05-02 21:13:34.057 Python[80806:3944025] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/pd/wynthhk510v_lw3l_4q234gh0000gn/T/org.python.python.savedState
2025-05-02 21:13:34.493 Python[80806:3944025] +[IMKClient subclass]: chose IMKClient_Legacy
2025-05-02 21:13:34.493 Python[80806:3944025] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


SystemExit: 

/Users/rav007/Library/Python/3.10/lib/python/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


: 

Brandon